# LangChain と PaLM API を組み合わせて利用する例

LangChain のパッケージをインストールします。

In [1]:
!pip install --user langchain==0.1.0 google-cloud-aiplatform==1.36.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


**注意：次のセルを実行する前にカーネルをリスタートしてください。**

## LangChain を用いて、基本的なパインプラインを実行する例

新製品の説明文から商品名のアイデアを3つ出してもらうためのプロンプトを定義します。

In [1]:
from langchain import PromptTemplate

template = """\
あなたは新製品の名前を考えるのが専門のコピーライターです。
新製品の印象的な名前の案を３つ考えてください。
３つの名前をカンマ区切りのリストで出力してください。
既存の特定の商品名は含めないでください。

次は製品の説明と出力の例です。
製品の説明: 子供向けの可愛いクレヨンセット
出力：クレヨンキッズ, クレヨンファン, クレヨンワールド

次の製品名を考えてください。
製品の説明：{description}
出力：
"""

prompt = PromptTemplate(template=template, input_variables=['description'])

テンプレートに対して、動的に文字列を埋め込むことができます。

In [2]:
print(prompt.format(description='若者向けのスマホケース'))

あなたは新製品の名前を考えるのが専門のコピーライターです。
新製品の印象的な名前の案を３つ考えてください。
３つの名前をカンマ区切りのリストで出力してください。
既存の特定の商品名は含めないでください。

次は製品の説明と出力の例です。
製品の説明: 子供向けの可愛いクレヨンセット
出力：クレヨンキッズ, クレヨンファン, クレヨンワールド

次の製品名を考えてください。
製品の説明：若者向けのスマホケース
出力：



LangChain のパイプラインから PaLM API を呼び出すクライアントオブジェクトを取得します。

In [3]:
from langchain_community.llms import VertexAI
llm = VertexAI(model_name='text-bison@002',
               temperature=0.4, max_output_tokens=128)

パイプラインを定義して実行します。

In [4]:
from langchain import LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
description = '若者向けの軽くてカラフルなスマホケース'
output = llm_chain.invoke({'description': description})
print(output)

{'description': '若者向けの軽くてカラフルなスマホケース', 'text': ' ポップケース, カラフルケース, ライトケース'}


得られた結果を Python のリストに変換します。

In [6]:
from langchain.output_parsers import CommaSeparatedListOutputParser
output_parser = CommaSeparatedListOutputParser()

In [7]:
output_parser.parse(output['text'])

['ポップケース', 'カラフルケース', 'ライトケース']

## pydantic を利用して、LLM からの応答をオブジェクトに変換する例

言語モデルからの応答を格納するオブジェクトを定義して、対応するパーサーを生成します。

In [8]:
from pydantic import BaseModel, Field

class ProductNames(BaseModel):
    setup: str = Field(description='product description')
    product_name1: str = Field(description='product name 1')
    product_name2: str = Field(description='product name 2')
    product_name3: str = Field(description='product name 3')

In [9]:
from langchain.output_parsers import PydanticOutputParser
parser = PydanticOutputParser(pydantic_object=ProductNames)

このパーサーは、指定のオブジェクトに対応した JSON 形式で応答を得るためのインストラクションが生成できます。

In [10]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"setup": {"title": "Setup", "description": "product description", "type": "string"}, "product_name1": {"title": "Product Name1", "description": "product name 1", "type": "string"}, "product_name2": {"title": "Product Name2", "description": "product name 2", "type": "string"}, "product_name3": {"title": "Product Name3", "description": "product name 3", "type": "string"}}, "required": ["setup", "product_name1", "product_name2", "product_name3"]}
```


パーサーが生成するインストラクションを含んだプロンプトを定義した後、パイプラインを再構成します。

In [11]:
template="""\
あなたは新製品の名前を考えるのが専門のコピーライターです。
新製品の印象的な名前の案を日本語で３つ考えてください。
既存の特定の商品名は含めないでください。

{format_instructions}

製品の説明: {description}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=['description'],
    partial_variables={
        'format_instructions': parser.get_format_instructions()}
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

パイプラインを実行して、得られた結果を ProductNames オブジェクトに変換します。

In [12]:
description = '象が踏んでも壊れないスマホケース'
output = llm_chain.invoke({'description': description})
parser.parse(output['text'])

ProductNames(setup='象が踏んでも壊れないスマホケース', product_name1='アイアンケース', product_name2='タフガード', product_name3='エレファントプロテクター')

## パーサーを含んだパイプラインを定義する例

TransformChain と SequentialChain を使用すると、パーサーを含んだ一連の処理をまとめてパイプラインとして定義できます。

In [13]:
from langchain.chains import TransformChain, SequentialChain

llm_chain = LLMChain(prompt=prompt, llm=llm, output_key='json_string')

def parse_output(inputs):
    text = inputs['json_string']
    return {'result': parser.parse(text)}

transform_chain = TransformChain(
    input_variables=['json_string'],
    output_variables=['result'],
    transform=parse_output
)

chain = SequentialChain(
    input_variables=['description'],
    output_variables=['result'],
    chains=[llm_chain, transform_chain],
)

In [14]:
description = '象が踏んでも壊れないスマホケース'
output = chain.invoke({'description': description})
output['result']

ProductNames(setup='象が踏んでも壊れないスマホケース', product_name1='アイアンケース', product_name2='タフガード', product_name3='エレファントプロテクター')